# Automatic Speech Recognition- ENGLISH


In [2]:
%pip install transformers
%pip install -U datasets
%pip install soundfile
%pip install librosa
%pip install gradio

%pip install ffmpeg-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --upgrade pip

The `librosa` library may need to have [ffmpeg](https://www.ffmpeg.org/download.html) installed.
- This page on [librosa](https://pypi.org/project/librosa/) provides installation instructions for ffmpeg.

In [3]:
from transformers.utils import logging
logging.set_verbosity_error()

c:\Users\etomras\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data preparation

In [ ]:
from datasets import load_dataset
dataset = load_dataset("librispeech_asr",
                       split="train.clean.100",
                       streaming=True,
                       trust_remote_code=True)

In [ ]:
example = next(iter(dataset))
dataset_head = dataset.take(3)
list(dataset_head)

In [ ]:
list(dataset_head)[2]

In [ ]:
example

In [ ]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(example["audio"]["array"],
             rate=example["audio"]["sampling_rate"])

### Build the pipeline

In [4]:
from transformers import pipeline
model_id = "distil-whisper/distil-small.en"
asr = pipeline(task="automatic-speech-recognition", model=model_id ) 
#"./models/distil-whisper/distil-small.en")

Info about [distil-whisper/distil-small.en](https://huggingface.co/distil-whisper)

In [5]:
asr.feature_extractor.sampling_rate

16000

In [ ]:
asr(example["audio"]["array"])

In [ ]:
example["text"]

### Building a shareable app with Gradio

* Testing with a longer audio file

In [6]:
import librosa

* Build the Gradio interface.

In [7]:
import os
import ffmpeg
import gradio as gr
demo = gr.Blocks()

In [8]:
def transcribe_long_form(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry.")
        return ""
    output = asr(
      filepath,
      max_new_tokens=256,
      chunk_length_s=30,
      batch_size=8,
    )
    return output["text"]

In [9]:
mic_transcribe = gr.Interface(
    fn=transcribe_long_form,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs=gr.Textbox(label="Transcription", lines=3),
    allow_flagging="never")

file_transcribe = gr.Interface(
    fn=transcribe_long_form,
    inputs=gr.Audio(sources="upload", type="filepath"),
    outputs=gr.Textbox(label="Transcription",lines=3),
    allow_flagging="never",
)

In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe,
         file_transcribe],
        ["Transcribe Microphone",
         "Transcribe Audio File"],
    )
#demo.launch(share=True, server_port=int(os.environ['PORT1']))

demo.launch(share=True, server_port=int(8000))

Running on local URL:  http://127.0.0.1:8000

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\etomras\AppData\Local\Programs\Python\Python39\lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 419, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\etomras\AppData\Local\Programs\Python\Python39\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\etomras\AppData\Local\Programs\Python\Python39\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\etomras\AppData\Local\Programs\Python\Python39\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\etomras\AppData\Local\Programs\Python\Python39\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "c:\U

In [ ]:
demo.close()

## Translation: Translate the English text into french and Vice-versa!!

In [ ]:
# Load model directly
from transformers import AutoProcessor, pipeline,AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_path=r"C:\\Users\\etomras\\SLOP\\SpeechToText\\RT\\models\\NLLB-200-distilled-600M"
#model_path=".\RT\models\nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

#device = "cuda:0" if torch.cuda.is_available() else "cpu"
#torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
#model.to(device)


In [ ]:
eng_text = """\
My puppy is adorable, \
Your kitten is cute.
Her panda is friendly.
His llama is thoughtful. \
We all have nice pets!"""

french_text = """\
Diminuez la vitesse du vent.
!"""

In [ ]:
#Examples: "en" for English, "fr" for French, "de" for German, "es" for Spanish, "zh" for Chinese, etc; translation_en_to_fr translates English to French
# You can view all the lists of languages here - https://huggingface.co/languages
translator_en_fr = pipeline("translation_en_to_fr", model=model,
                        tokenizer=tokenizer,
    max_new_tokens=128,
    batch_size=16,
    torch_dtype=torch.bfloat16)

In [ ]:
text_translated_in_fr = translator_en_fr(eng_text, src_lang="eng_Latn", tgt_lang="fra_Latn")
print(text_translated_in_fr)

In [ ]:
#Examples: "en" for English, "fr" for French, "de" for German, "es" for Spanish, "zh" for Chinese, etc; translation_en_to_fr translates English to French
# You can view all the lists of languages here - https://huggingface.co/languages
translator_fr_en = pipeline("translation_fr_to_en", model=model,
                        tokenizer=tokenizer,
    max_new_tokens=128,
    batch_size=16,
    torch_dtype=torch.bfloat16)

text_translated_in_en = translator_fr_en(french_text,src_lang="fra_Latn", tgt_lang="eng_Latn", )
text_translated_in_en